In [17]:
# ! pip install tqdm

In [1]:
# Libraries
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
# from hda import * # this is the PYTHON version

import numpy as np
import pandas as pd
from tqdm import tqdm

In [8]:
#Sea depth data
sea_d = 'dYear/dYear'
sea_d = xr.open_dataset(sea_d)
sea_d
# Three dimensions matrix that contains the depth of the sea water 
sea_d = sea_d['model_depth']
sea_d_df = sea_d.to_dataframe(name= None, dim_order=None)
sea_d_df = sea_d_df[~sea_d_df['model_depth'].isnull()]
sea_d_df.to_csv("sea_depth_final.csv")
# sea_d_df[(sea_d_df["model_depth"]>50) & (sea_d_df["model_depth"]<500)]
# sea_d_df

In [4]:
min(sea_d_df["model_depth"])

12.82080078125

In [2]:
#Sea water

#Northward sea water speed
sea_v_n = 'xYear/xYear'
sea_v_n_xr = xr.open_dataset(sea_v_n)
# sea_v_n_xr
# Four dimensions matrix that contains the speed in North
sea_v_n = sea_v_n_xr['u']

#Eastward sea water speed
sea_v_e = 'yYear/yYear'
sea_v_e_xr = xr.open_dataset(sea_v_e)
# sea_v_e_xr
# Four dimensions matrix that contains the speed in East
sea_v_e = sea_v_e_xr['v']

# Net speed
sea_v = np.sqrt(np.power(sea_v_e,2)+np.power(sea_v_n,2)) 
# sea_v
sea_v_df = sea_v.to_dataframe(name= "speed", dim_order=None)
sea_v_df = sea_v_df[~sea_v_df["speed"].isnull()]



In [3]:
sea_v_df

latitude  longitude     speed
time       depth y       x                                    
2019-01-01 0.0   -28.500 3.375  64.641876 -38.246426  0.116099
                         3.500  64.629074 -37.998734  0.122194
                         3.625  64.615822 -37.751301  0.124140
                         3.750  64.602112 -37.504143  0.120675
                         3.875  64.587936 -37.257263  0.114130
...                                   ...        ...       ...
2019-12-31 0.0   -26.750 4.625  65.972496 -35.190697  0.083837
                         4.750  65.953957 -34.930939  0.152007
                 -26.625 4.500  66.096512 -35.406864  0.025563
                         4.625  66.078362 -35.145546  0.060635
                         4.750  66.059731 -34.884640  0.107250

[51830 rows x 3 columns]

In [5]:
sea_v_df.to_csv("sea_v_final_dt2.csv")

In [9]:
# Depth fetching 

depth = pd.read_csv("sea_depth_final.csv", delimiter=',', decimal='.')
sea_v = pd.read_csv("sea_v_final_dt2.csv", delimiter=',', decimal='.')

sea_depth = []
counter = 0
for row in tqdm(range(len(sea_v))): 
    value = depth[(depth["latitude"] == sea_v["latitude"][row]) & (depth["longitude"] == sea_v["longitude"][row])]["model_depth"].tolist()
    if len(value) !=0:
        sea_depth += value
    else: 
        counter += 1
# counter

sea_v["sea_depth"] = sea_depth 
# sea_v
# sea_v.to_csv("Speed_depth_filter.csv")

sea_v_filter = sea_v[(sea_v["sea_depth"]>40) & (sea_v["sea_depth"]<200)]
# sea_v_filter.to_csv("Speed_depth_filter.csv")

sea_v_filter = sea_v_filter.reset_index()


100%|██████████| 51830/51830 [00:44<00:00, 1171.55it/s]


In [6]:
# sea_v = pd.read_csv("sea_v_final_dt2.csv",delimiter=',', decimal='.')

In [15]:
# Power calculation 
def tidalPower(n = 1, dens = 1025, area = 10, v = 1, maxV = 5):
    """
    P = Power generated in W
    n = turbine efficiency [0-1]
    dens = water density in kg/m^3
    area = sweep area of the turbine in m^2
    v = velocity of the flow in m/s
    """
    if v < maxV:
        P = n*dens*area*v**3/2
    else:
        P = 0
    return P

def wavesPower(H = 5, Te = 10, dens = 1025, g = 9.8, maxH = 8):
    """
    P  = wave energy flux per unit of wave-crest length
    H = the significant wave height in m
    Te = the wave energy period in s
    dens = the water density
    g  = the acceleration by gravity
    
    When the significant wave height is given in meters, 
    and the wave period in seconds, the result is the wave power in kilowatts (kW) 
    per meter of wave front length.
    """
    if H < maxH:
        P = dens*g**2*H**2*Te/64*3.14
    else:
        P = 0
    return P
    
def energyProduced(passedTime = None, power = None):
    e = power/3600*passedTime
    return e

In [16]:
# Possible power calculation 
nTurbine = 0.4
areaTurbine = 5**2 * 3.14
# velocity = 1.5
out_energy = []

for row in tqdm(range(len(sea_v_filter))):
    velocity = sea_v_filter["speed"][row]
    totalEnergy = 0
    for i in range(24): #in 24 hours
        P = tidalPower(n = nTurbine, area = areaTurbine, v = velocity)
        energy = energyProduced(3600, P)
        totalEnergy = (totalEnergy + energy)
        #print(P/1000)
    out_energy += [totalEnergy]

100%|██████████| 7665/7665 [00:00<00:00, 8212.19it/s]


In [17]:
sea_v_filter["energy"] = out_energy
sea_v_filter.to_csv("Energy_per_day_final.csv")

In [29]:
max_energy = max(sea_v_filter["energy"])

In [30]:
sea_v_filter[sea_v_filter["energy"]== max_energy]

,index,time,depth,y,x,latitude,longitude,speed,sea_depth,energy
63938,519373,2019-01-19,0.0,-27.625,3.75,65.34461,-37.269547,0.658189,174.60449,110125.176912


In [31]:
sea_v_filter["latitude"][63938]

65.34461

In [35]:
max_value = sea_v_filter[(sea_v_filter["latitude"]==sea_v_filter["latitude"][63938]) & (sea_v_filter["longitude"]==sea_v_filter["longitude"][63938])]
total_energy = max_value["energy"].mean()
total_energy

29886.144695771756